In [9]:
from OCC.Core.gp import gp_Vec, gp_Trsf
from OCC.Core.BRepPrimAPI import BRepPrimAPI_MakeBox
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Fuse
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_Transform

In [10]:
def build_i_section(length, flange_width, section_height, flange_thick, web_thick):
    # Create a basic I-section using boxes for web and flanges.
    # The I-section is defined by its length, flange width, section height, flange thickness, and web thickness.
    web_height = section_height - 2 * flange_thick

    # Bottom flange
    bottom = BRepPrimAPI_MakeBox(length, flange_width, flange_thick).Shape()

    # Top flange (placed at the top of the section)
    top = BRepPrimAPI_MakeBox(length, flange_width, flange_thick).Shape()
    move_top = gp_Trsf()
    move_top.SetTranslation(gp_Vec(0, 0, section_height - flange_thick))
    top = BRepBuilderAPI_Transform(top, move_top).Shape()

    # Web (centered between the flanges)
    web = BRepPrimAPI_MakeBox(length, web_thick, web_height).Shape()
    move_web = gp_Trsf()
    move_web.SetTranslation(gp_Vec(0, (flange_width - web_thick) / 2, flange_thick))
    web = BRepBuilderAPI_Transform(web, move_web).Shape()

    # Fuse all parts into one solid I-section
    section = BRepAlgoAPI_Fuse(bottom, top).Shape()
    section = BRepAlgoAPI_Fuse(section, web).Shape()

    return section

In [11]:
def combine_two_i_sections(base_section, gap_between):
    # Duplicate the I-section and place it parallel to the first.

    move = gp_Trsf()
    move.SetTranslation(gp_Vec(gap_between, 0, 0))
    second_section = BRepBuilderAPI_Transform(base_section, move).Shape()

    # Merge both sections together
    combined = BRepAlgoAPI_Fuse(base_section, second_section).Shape()

    return combined

In [12]:
def attach_end_battens(structure, batten_width=430, batten_height=300, batten_thickness=10, gap=450):
    # Attach battens to the ends of the combined I-sections.
    # The battens are rectangular prisms that connect the two I-sections.
    # Add battens at both ends to connect the I-sections.
    
    batten = BRepPrimAPI_MakeBox(batten_thickness, batten_width, batten_height).Shape()

    # Left batten
    move_left = gp_Trsf()
    move_left.SetTranslation(gp_Vec(-batten_thickness, -batten_width / 2, 0))
    batten_left = BRepBuilderAPI_Transform(batten, move_left).Shape()

    # Right batten
    move_right = gp_Trsf()
    move_right.SetTranslation(gp_Vec(gap, -batten_width / 2, 0))
    batten_right = BRepBuilderAPI_Transform(batten, move_right).Shape()

    # Merge battens to the structure
    structure = BRepAlgoAPI_Fuse(structure, batten_left).Shape()
    structure = BRepAlgoAPI_Fuse(structure, batten_right).Shape()

    return structure

In [13]:
def add_diagonal_lacing(structure, flat_width=100, flat_thickness=8, column_gap=450, column_height=6000):
    # Add diagonal lacing bars between the two I-sections.
    # The lacing bars are flat bars that connect the two I-sections diagonally.
    # Add lacing bars (approx 45°) between the I-sections.

    lacing_bar = BRepPrimAPI_MakeBox(flat_thickness, flat_width, column_height).Shape()

    # Move lacing bar to mid-height between the two I-sections
    move_lace = gp_Trsf()
    move_lace.SetTranslation(gp_Vec(column_gap / 2, -flat_width / 2, column_height / 2))
    lacing_bar = BRepBuilderAPI_Transform(lacing_bar, move_lace).Shape()

    structure = BRepAlgoAPI_Fuse(structure, lacing_bar).Shape()

    return structure

In [14]:
def make_laced_column():
    # Basic parameters
    column_length = 6000
    section_width = 200
    section_height = 450
    flange_thickness = 10
    web_thickness = 6
    spacing_between_sections = 450

    single_section = build_i_section(column_length, section_width, section_height, flange_thickness, web_thickness)

    double_section = combine_two_i_sections(single_section, spacing_between_sections)

    framed_column = attach_end_battens(double_section)

    final_structure = add_diagonal_lacing(framed_column)

    return final_structure

In [15]:
from OCC.Display.SimpleGui import init_display

In [16]:
if __name__ == "__main__":
    display, start_display, add_menu, add_function_to_menu = init_display()

    compound_column = make_laced_column()

    display.DisplayShape(compound_column, update=True)
    display.FitAll()
    start_display()